<a href="https://colab.research.google.com/github/fedelosio99/Mioti/blob/main/Proyecto%20Final/Scraping_Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import requests
import time

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
def scrape_category_stats(team_url, category,Soup):
    global request_count, start_time, MAX_REQUESTS_PER_MINUTE, TIME_LIMIT
    soup = Soup  # Parse with BeautifulSoup
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and f'all_comps/{category}' in l]

    if links:
        data = requests.get(f"https://fbref.com{links[0]}")
        html_data = StringIO(data.text)
        #if category is defense the category tittle should be defensive actions and if its keeper it should be goalkeeping
        if category == 'keeper':
            category = 'Goalkeeping'
        if category == 'defense':
            category = 'Defensive Actions'
        else:
          category= category.capitalize()

        category_data = pd.read_html(html_data, match=category)[0]
        category_data.columns = category_data.columns.droplevel()
        return category_data
    # If no links found, return an empty DataFrame to avoid NoneType error
    return none # Return empty DataFrame

In [5]:
a='passing'
a.capitalize()

'Passing'

In [6]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [7]:
data = requests.get(standings_url)

In [9]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links

['/en/squads/822bd0ba/Liverpool-Stats',
 '/en/matches/1f604fbd/Liverpool-Leicester-City-December-26-2024-Premier-League',
 '/en/matches/c1a66ac0/West-Ham-United-Liverpool-December-29-2024-Premier-League',
 '/en/matches/4cef863f/North-West-Derby-Liverpool-Manchester-United-January-5-2025-Premier-League',
 '/en/matches/118f8df8/Nottingham-Forest-Liverpool-January-14-2025-Premier-League',
 '/en/matches/5e8445c1/Brentford-Liverpool-January-18-2025-Premier-League',
 '/en/players/e342ad68/Mohamed-Salah',
 '/en/players/7a2e46a8/Alisson',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/matches/668dad03/Arsenal-Ipswich-Town-December-27-2024-Premier-League',
 '/en/matches/7e6892e4/Brentford-Arsenal-January-1-2025-Premier-League',
 '/en/matches/52186da4/Brighton-and-Hove-Albion-Arsenal-January-4-2025-Premier-League',
 '/en/matches/ee9ce5e2/North-London-Derby-Arsenal-Tottenham-Hotspur-January-15-2025-Premier-League',
 '/en/matches/1fdaaaba/Arsenal-Aston-Villa-January-18-2025-Premier-League',
 '/en/pla

In [10]:
links = [l for l in links if '/squads/' in l]

In [11]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [12]:
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [13]:
def request_limit_control():
    global request_count, start_time, MAX_REQUESTS_PER_MINUTE, TIME_LIMIT

    # Calculate the elapsed time since the first request of the current minute
    elapsed_time = time.time() - start_time

    # If the number of requests exceeds the limit, wait until the next minute
    if request_count >= MAX_REQUESTS_PER_MINUTE:
        if elapsed_time < TIME_LIMIT:
            sleep_time = TIME_LIMIT - elapsed_time
            #print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
            time.sleep(sleep_time)
        # Reset the start time and request count for the next minute
        start_time = time.time()
        request_count = 0

    # Increment the request count after the check
    request_count += 1

In [14]:
import requests
from requests.exceptions import RequestException, HTTPError, Timeout

def safe_request(url):
    try:
        # Attempt to send the request and check the status code
        response = requests.get(url, timeout=60)  # Set a timeout (e.g., 10 seconds)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
        return response  # Return the response if successful
    except Timeout:
        print(f"Request timed out for {url}")
        return None
    except HTTPError as http_err:
        print(f"HTTP error occurred for {url}: {http_err}")
        return None
    except RequestException as req_err:
        print(f"Request failed for {url}: {req_err}")
        return None


In [15]:
def extract_table(html_data, pattern):
    try:
      if  pattern == 'Scores & Fixtures':
        table = pd.read_html(html_data, match=pattern)[0]
        return table
      else:
        table = pd.read_html(html_data, match=pattern)[0]
        table.columns = table.columns.droplevel()  # Si las columnas tienen múltiples niveles
        return table
    except ValueError:
        print(f"Error: No se encontró la tabla con el patrón '{pattern}'.")
        return None
    except Exception as e:
        print(f"Error inesperado al procesar la tabla '{pattern}': {e}")
        return None

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Codigo para extraer la data de todos los jugadores del 2016 al 2024
years = range(2024, 2016, -1)
competitions = [
    {"name": "Premier League", "url": "https://fbref.com/en/comps/9/Premier-League-Stats"},
    {"name": "La Liga", "url": "https://fbref.com/en/comps/12/La-Liga-Stats"},
    {"name": "Serie A", "url": "https://fbref.com/en/comps/11/Serie-A-Stats"},
    {"name": "Bundesliga", "url": "https://fbref.com/en/comps/20/Bundesliga-Stats"},
    {"name": "Ligue 1", "url": "https://fbref.com/en/comps/13/Ligue-1-Stats"},
    {"name": "Eredivisie", "url": "https://fbref.com/en/comps/23/Eredivisie-Stats"},
    {"name": "Primeira Liga", "url": "https://fbref.com/en/comps/32/Primeira-Liga-Stats"},
    {"name": "Liga profesional argentina", "url": "https://fbref.com/en/comps/21/Liga-Profesional-Argentina-Stats"},
    # Add more competitions as needed
]

all_matches = []
all_playerstats = []
all_goalkeepingstats = []
all_defense = []
all_shooting = []
all_passing = []
all_goal_and_shots = []

# Request limit control
MAX_REQUESTS_PER_MINUTE = 10
TIME_LIMIT = 60  # Time limit in seconds (1 minute)
request_count = 0  # Counter for tracking the number of requests
start_time = time.time()  # Start time for the first request

# Loop through each competition
for competition in competitions:
    standings_url = competition["url"]
    for year in years:
        print(f"Scraping data for {competition['name']} in {year}")
        # Request the standings page for each competition
        request_limit_control()
        data = safe_request(standings_url)
        soup = BeautifulSoup(data.text, 'html.parser')  # Parse HTML

        # Get the teams' links from the standings table
        standings_table = soup.select('table.stats_table')[0]
        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        # Get the URL for the previous season
        previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"

        # Loop through each team's page to get match data
        for team_url in team_urls:
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

            # Get the team's match data
            request_limit_control()
            data = safe_request(team_url)
            soup = BeautifulSoup(data.text, 'html.parser')
            html_data = StringIO(data.text)
            try:
                matches = extract_table(html_data, "Scores & Fixtures")
                playerstats = extract_table(html_data, "Standard Stats")
                goalkeepingstats = extract_table(html_data, "Goalkeeping")
                playersshooting = extract_table(html_data, "Shooting")
                playerspassing = extract_table(html_data, "Passing")
                playersdefense = extract_table(html_data, "Defensive Actions")
                playersgoalandshots = extract_table(html_data, "Goal and Shot Creation")

                # Validamos si alguna tabla no se pudo extraer
                if matches is None or matches.empty:
                    print("La tabla 'Scores & Fixtures' no se pudo extraer o está vacía.")
                if playerstats is None or playerstats.empty:
                    print("La tabla 'Standard Stats' no se pudo extraer o está vacía.")
                if goalkeepingstats is None or goalkeepingstats.empty:
                    print("La tabla 'Goalkeeping' no se pudo extraer o está vacía.")
                if playersshooting is None or playersshooting.empty:
                    print("La tabla 'Shooting' no se pudo extraer o está vacía.")
                if playerspassing is None or playerspassing.empty:
                    print("La tabla 'Passing' no se pudo extraer o está vacía.")
                if playersdefense is None or playersdefense.empty:
                    print("La tabla 'Defensive Actions' no se pudo extraer o está vacía.")
                if playersgoalandshots is None or playersgoalandshots.empty:
                    print("La tabla 'Goal and Shot Creation' no se pudo extraer o está vacía.")

                matches = matches.copy()
                matches.loc[:, "Season"] = year
                matches.loc[:, "Team"] = team_name

                playerstats = playerstats.copy()
                playerstats.loc[:, "Season"] = year
                playerstats.loc[:, "Team"] = team_name

                goalkeepingstats = goalkeepingstats.copy()
                goalkeepingstats.loc[:, "Season"] = year
                goalkeepingstats.loc[:, "Team"] = team_name

                playersshooting = playersshooting.copy()
                playersshooting.loc[:, "Season"] = year
                playersshooting.loc[:, "Team"] = team_name

                playerspassing = playerspassing.copy()
                playerspassing.loc[:, "Season"] = year
                playerspassing.loc[:, "Team"] = team_name

                playersgoalandshots = playersgoalandshots.copy()
                playersgoalandshots.loc[:, "Season"] = year
                playersgoalandshots.loc[:, "Team"] = team_name

                playersdefense = playersdefense.copy()
                playersdefense.loc[:, "Season"] = year
                playersdefense.loc[:, "Team"] = team_name

                # Append the data to the list of all matches
                all_matches.append(matches)
                all_playerstats.append(playerstats)
                all_goalkeepingstats.append(goalkeepingstats)
                all_defense.append(playersdefense)
                all_shooting.append(playersshooting)
                all_passing.append(playerspassing)
                all_goal_and_shots.append(playersgoalandshots)

            except Exception as e:
                print(f"Error general en el proceso: {e}")

# Optionally, combine all matches into a single DataFrame
all_matches_df = pd.concat(all_matches, ignore_index=True)
playerstats_df = pd.concat(all_playerstats, ignore_index=True)
goalkeepingstats_df = pd.concat(all_goalkeepingstats, ignore_index=True)
defense_df = pd.concat(all_defense, ignore_index=True)
shooting_df = pd.concat(all_shooting, ignore_index=True)
passing_df = pd.concat(all_passing, ignore_index=True)
goal_and_shots_df = pd.concat(all_goal_and_shots, ignore_index=True)


Scraping data for Premier League in 2024
Scraping data for Premier League in 2023
Scraping data for Premier League in 2022
Scraping data for Premier League in 2021
Scraping data for Premier League in 2020
Scraping data for Premier League in 2019
Scraping data for Premier League in 2018
Scraping data for Premier League in 2017
Scraping data for La Liga in 2024
Scraping data for La Liga in 2023
Scraping data for La Liga in 2022
Scraping data for La Liga in 2021
Scraping data for La Liga in 2020
Scraping data for La Liga in 2019
Scraping data for La Liga in 2018
Scraping data for La Liga in 2017
Scraping data for Serie A in 2024
Scraping data for Serie A in 2023
Scraping data for Serie A in 2022
Scraping data for Serie A in 2021
Scraping data for Serie A in 2020
Scraping data for Serie A in 2019
Scraping data for Serie A in 2018
Scraping data for Serie A in 2017
Scraping data for Bundesliga in 2024
Scraping data for Bundesliga in 2023
Scraping data for Bundesliga in 2022
Scraping data for

In [ ]:
all_matches_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/all_matches_df.csv')
playerstats_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/playerstats_df.csv')
goalkeepingstats_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/goalkeepingstats_df.csv')
defense_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/defense_df.csv')
shooting_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/shooting_df.csv')
passing_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/passing_df.csv')
goal_and_shots_df.to_csv('/content/drive/MyDrive/Master Ds/Proyecto Final/goal_and_shots_df.csv')


In [18]:
# Codigo para extraer la data de todos los jugadores del 2016 al 2024
years = range(2024,2022, -1)
competitions = [
    {"name": "Premier League", "url": "https://fbref.com/en/comps/9/Premier-League-Stats"},
    {"name": "La Liga", "url": "https://fbref.com/en/comps/12/La-Liga-Stats"},
    {"name": "Serie A", "url": "https://fbref.com/en/comps/11/Serie-A-Stats"},
    {"name": "Bundesliga", "url": "https://fbref.com/en/comps/20/Bundesliga-Stats"},
    {"name": "Ligue 1", "url": "https://fbref.com/en/comps/13/Ligue-1-Stats"},
    {"name": "Eredivisie", "url": "https://fbref.com/en/comps/23/Eredivisie-Stats"},
    {"name": "Primeira Liga", "url": "https://fbref.com/en/comps/32/Primeira-Liga-Stats"},
    {"name": "Liga profesional argentina", "url": "https://fbref.com/en/comps/21/Liga-Profesional-Argentina-Stats"},
    # Add more competitions as needed
]

all_matches = []
shotting_matches = []
goalkeeping_matches = []
passing_matches = []
pass_types_matches = []
goal_and_shots_matches = []
defense_matches = []
possession_matches = []
miscalleneous_matches = []


# Request limit control
MAX_REQUESTS_PER_MINUTE = 10
TIME_LIMIT = 60  # Time limit in seconds (1 minute)
request_count = 0  # Counter for tracking the number of requests
start_time = time.time()  # Start time for the first request

# Loop through each competition
for competition in competitions:
    standings_url = competition["url"]
    for year in years:
        print(f"Scraping data for {competition['name']} in {year}")
        # Request the standings page for each competition
        request_limit_control()
        data = safe_request(standings_url)
        soup = BeautifulSoup(data.text, 'html.parser')  # Parse HTML

        # Get the teams' links from the standings table
        standings_table = soup.select('table.stats_table')[0]
        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        # Get the URL for the previous season
        previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"

        # Loop through each team's page to get match data
        for team_url in team_urls:

            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

            # Get the team's match data
            request_limit_control()
            data = safe_request(team_url)
            soup = BeautifulSoup(data.text, 'html.parser')
            html_data = StringIO(data.text)
            try:
                matches = extract_table(html_data, "Scores & Fixtures")
                 # shooting data
                temp_soup = BeautifulSoup(data.text)

                # filter
                temp_links = [l.get("href") for l in temp_soup.find_all('a')]
                temp_links = [l for l in temp_links if 'all_comps/shooting/' in l]

                shooting_data = requests.get(f"https://fbref.com{temp_links[0]}")
                request_limit_control()
                shooting = pd.read_html(shooting_data.text, match="Shooting")[0]

                # drop first unneeded level
                shooting.columns = shooting.columns.droplevel()

            except Exception as e:
                print(f"Error general en el proceso: {e}")

# Optionally, combine all matches into a single DataFrame



Scraping data for Premier League in 2024
Error general en el proceso: argument of type 'NoneType' is not iterable
Error general en el proceso: argument of type 'NoneType' is not iterable
Error general en el proceso: argument of type 'NoneType' is not iterable
Error general en el proceso: argument of type 'NoneType' is not iterable
Error general en el proceso: argument of type 'NoneType' is not iterable


KeyboardInterrupt: 